In [1]:
import sys 
sys.path.append('../')
import robust_laplacian
import jax 
import jax.numpy as np 
import numpy as onp
from jax.random import normal
from jax import grad, hessian, vmap, pmap
from jax.flatten_util import ravel_pytree
from jax.lax import scan
from functools import partial
import jax.random as jax_random
import matplotlib.pyplot as plt 
from IPython.display import clear_output
# import scipy.sparse.linalg as sla
# import jax.scipy.sparse.linalg as sla

import jaxopt

import polyscope as ps
from mesh_distributions import BunnyMesh, DownSampledBunny, MeshDistr, SphereMesh
from ergodic_mmd.planner import ErgodicMMDPlanner
from ergodic_mmd.kernel import RBF


objc[22442]: Class GLFWWindowDelegate is implemented in both /Users/ia285/miniconda3/lib/python3.10/site-packages/polyscope_bindings.cpython-310-darwin.so (0x137646e90) and /Users/ia285/miniconda3/lib/python3.10/site-packages/open3d/cpu/pybind.cpython-310-darwin.so (0x16995adb0). One of the two will be used. Which one is undefined.
objc[22442]: Class GLFWApplicationDelegate is implemented in both /Users/ia285/miniconda3/lib/python3.10/site-packages/polyscope_bindings.cpython-310-darwin.so (0x137646e68) and /Users/ia285/miniconda3/lib/python3.10/site-packages/open3d/cpu/pybind.cpython-310-darwin.so (0x16995ae28). One of the two will be used. Which one is undefined.
objc[22442]: Class GLFWContentView is implemented in both /Users/ia285/miniconda3/lib/python3.10/site-packages/polyscope_bindings.cpython-310-darwin.so (0x137646ee0) and /Users/ia285/miniconda3/lib/python3.10/site-packages/open3d/cpu/pybind.cpython-310-darwin.so (0x16995ae50). One of the two will be used. Which one is undefin

In [6]:
bunny_mesh = SphereMesh()
mesh = SphereMesh()

info_distr = lambda x: np.exp(-np.sum((x-0.75)**2))
mesh_func = MeshDistr(bunny_mesh, info_distr)
ds_mesh_func = MeshDistr(mesh, info_distr)

vect_info = vmap(info_distr, in_axes=(0,))

In [7]:
X_samples = ds_mesh_func.verts
P_XI      = ds_mesh_func.func_vals
P_XI = P_XI/np.sum(P_XI)
h = 0.01
args = {'h' : h, 'X_samples' : X_samples, 'P_XI' : P_XI}

In [8]:
planner = ErgodicMMDPlanner(ds_mesh_func, RBF)

X = planner.plan(args, max_iter=1)

In [10]:
ps.init()

ps_bunny_mesh = ps.register_point_cloud("bunny", bunny_mesh.verts)
ps_bunny_mesh.set_transparency(0.8)
ps_bunny_mesh.add_scalar_quantity('info_distr', mesh_func.func_vals, cmap='blues')

ps_traj         = ps.register_curve_network("trajectory", X * 1.25, edges="line")

for _ in range(1000):
    X = planner.plan(args, max_iter=1)
    ps_traj.update_node_positions(X*1.25)
    ps.frame_tick()

    plt.pause(0.001)


# Render stills